<a href="https://colab.research.google.com/github/micheleguidaa/robust-organ-scheduler/blob/main/robust_organ_scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pianificazione Robusta dell'Orario di Prelievo in Interventi di Trapianto

**Studente:** Michele Guida  
**Corso:** Decision Support Systems  
**Repository:** [github.com/micheleguidaa/robust-organ-scheduler](https://github.com/micheleguidaa/robust-organ-scheduler)

## 1. Definizione del Problema

Il progetto affronta la scelta dell'orario di inizio ($t_{start}$) di un intervento di prelievo multi-organo da un singolo donatore.
L'obiettivo è minimizzare i ritardi di partenza dei mezzi di trasporto assegnati ai diversi organi, tenendo conto
della diversa importanza clinica e dell'incertezza sulla durata dell'intervento.

### Modello Matematico

Sia $\mathbf{T}$ l'insieme discreto degli orari di inizio possibili e $\mathbf{D} = \{d_{min}, d_{med}, d_{max}\}$ l'insieme delle possibili durate dell'intervento.

Definiamo il **tempo di attesa** $\delta_i$ per l'organo $i$ come il tempo che intercorre tra la disponibilità dell'organo (fine intervento) e la prima partenza utile del mezzo di trasporto $S_i^j$:

$$
\delta_{i}(t_{start},d) = \min_{j: S_{i}^{j} \ge t_{start}+d} (S_{i}^{j} - (t_{start}+d))
$$

La **funzione di costo** (ritardo totale pesato) per uno scenario è data da:

$$
F(t_{start},d) = \sum_{i \in \mathbf{I}} w_{i} \cdot \delta_{i}(t_{start},d)
$$

### Obiettivo: Ottimizzazione Robusta (Minimax)
Poiché la durata $d$ è incerta, adottiamo un approccio robusto che minimizza il costo nel *caso peggiore* (worst-case scenario):

$$
t_{start}^* = \arg \min_{t_{start} \in \mathbf{T}} \left( \max_{d \in \mathbf{D}} F(t_{start}, d) \right)
$$

## 2. Soluzione e Implementazione dell'Algoritmo

### Descrizione dell'Approccio
La funzione `robust_organ_scheduler` implementa una strategia di **enumerazione esplicita** per risolvere il problema di ottimizzazione robusta (Minimax). Dato che gli insiemi degli orari di inizio ($\mathbf{T}$), delle durate ($\mathbf{D}$) e delle partenze ($\mathbf{S}_i$) sono discreti e finiti, l'algoritmo esplora l'intero spazio delle soluzioni ammissibili.

Il flusso logico è il seguente:
1.  **Ciclo Esterno (Decisione):** Itera su tutti i possibili orari di inizio $t_{start} \in \mathbf{T}$.
2.  **Ciclo Interno (Incertezza):** Per ogni $t_{start}$, valuta tutti gli scenari di durata $d \in \mathbf{D}$ per identificare il *caso peggiore* (worst-case).
3.  **Calcolo del Costo:** Per ogni combinazione $(t_{start}, d)$, calcola il ritardo pesato sommando le attese $\delta_i$ di ciascun organo rispetto alla prima partenza utile disponibile.
4.  **Selezione Ottima:** Seleziona il $t_{start}$ che minimizza il costo del caso peggiore trovato.

### Analisi della Complessità Computazionale
La complessità dell'algoritmo dipende dalla cardinalità degli insiemi in input.
Definiamo:
* $|\mathbf{T}|$: numero di orari di inizio possibili.
* $|\mathbf{D}|$: numero di scenari di durata (nel nostro caso $K=3$).
* $ N $: numero di organi (nel nostro caso $N=5$).
* $|\mathbf{S}_i|$: numero di orari di partenza disponibili per l'organo $i$.

L'algoritmo esegue due cicli annidati principali. Al livello più interno, per ogni organo, viene eseguita una scansione lineare delle partenze disponibili per trovare la prima valida (`min(partenze_valide)`).

La complessità temporale asintotica è dunque:

$$
O\left( |\mathbf{T}| \cdot |\mathbf{D}| \cdot \sum_{i=1}^{N} |\mathbf{S}_i| \right)
$$

Poiché $|\mathbf{D}|$ e $N$ sono costanti molto piccole e fisse nel problema specifico, la complessità scala linearmente rispetto al numero di orari di inizio da valutare ($|\mathbf{T}|$) e al volume totale dei voli disponibili ($\sum |\mathbf{S}_i|$).

In [1]:
import numpy as np
from datetime import datetime, date, timedelta
from typing import Sequence, Mapping, Tuple, Optional

def parse_orario(s: str) -> datetime:
    """
    Converte una stringa "HH:MM" in un oggetto datetime riferito alla data odierna
    """
    t = datetime.strptime(s, "%H:%M").time()
    return datetime.combine(date.today(), t)

def robust_organ_scheduler(
    orari_possibili_inizio_intervento: Sequence[datetime],
    durate_possibili_intervento: Sequence[timedelta],
    organi: Sequence[str],
    orari_partenza: Mapping[str, Sequence[datetime]],
    pesi: Mapping[str, float],
) -> Tuple[float, Optional[datetime], timedelta]:
    """
    Risolve il problema di scheduling robusto (minimax) restituendo:
    - valore della funzione obiettivo (float)
    - orario di inizio intervento ottimo (datetime) oppure None se non esiste soluzione
    - tempo di esecuzione dell'algoritmo (timedelta)
    """
    # Avvio del timer per misurare il tempo di esecuzione dell’algoritmo.
    inizio_esecuzione_algoritmo = datetime.now()

    # Inizializzazione del problema minimax: vogliamo minimizzare il massimo delle attese pesate
    funzione_obiettivo = np.inf
    migliore_orario_inizio_intervento = None

    # Valutazione di ogni possibile orario di inizio intervento
    for orario_inizio_intervento in orari_possibili_inizio_intervento:

        # Per questo orario consideriamo il peggior scenario tra tutte le durate
        somma_pesata_attese_max = -np.inf

        for durata_intervento in durate_possibili_intervento:
            # Calcolo della somma pesata delle attese per questa durata
            somma_pesata_attese = 0.0

            for organo in organi:
                # Orario di fine intervento per questa durata
                orario_fine_intervento = orario_inizio_intervento + durata_intervento

                # Selezioniamo solo le partenze non antecedenti alla fine dell'intervento
                partenze_valide = [
                    orario_partenza for orario_partenza in orari_partenza[organo]
                    if orario_partenza >= orario_fine_intervento
                ]

                # Se non ci sono partenze valide l’intervento diventa impossibile → costo infinito
                if not partenze_valide:
                    somma_pesata_attese = np.inf
                    break

                # Prima partenza disponibile
                prima_partenza_utile = min(partenze_valide)

                # Tempo di attesa rispetto alla fine dell’intervento
                attesa = prima_partenza_utile - orario_fine_intervento
                attesa_minuti = attesa.total_seconds() / 60

                # Aggiunta dell’attesa pesata secondo l’importanza dell’organo
                somma_pesata_attese += pesi[organo] * attesa_minuti

            # Aggiornamento dello scenario peggiore (max) per questo t_start
            if somma_pesata_attese > somma_pesata_attese_max:
                somma_pesata_attese_max = somma_pesata_attese

        # Aggiorniamo il best t_start globale confrontando i valori worst-case ottenuti
        if somma_pesata_attese_max < funzione_obiettivo:
            funzione_obiettivo = somma_pesata_attese_max
            migliore_orario_inizio_intervento = orario_inizio_intervento

    # Fine della misurazione del tempo totale di esecuzione.
    fine_esecuzione_algoritmo = datetime.now()

    # tempo_di_esecuzione quantifica il costo computazionale dell’enumerazione.
    tempo_di_esecuzione = fine_esecuzione_algoritmo - inizio_esecuzione_algoritmo

    return funzione_obiettivo, migliore_orario_inizio_intervento, tempo_di_esecuzione


## 3. Test Illustrativo su Piccola Istanza

In questa sezione applichiamo l'algoritmo su un'istanza di dimensioni ridotte per verificarne la correttezza logica prima di passare ai test di performance su larga scala.

In [2]:
# --- CONFIGURAZIONE DATI DI TEST ---

# 1. Pesi clinici come da specifiche di progetto
pesi = {
    "Cuore": 100,
    "Polmoni": 90,
    "Rene Sx": 20,
    "Rene Dx": 20,
    "Fegato": 20
}
organi = list(pesi.keys())

# 2. Durate dell'intervento (Incertezza)
# Ipotizziamo: min 3 ore (180 min), max 5 ore (300 min)
d_min = 180
d_max = 300
d_med = (d_min + d_max) / 2

durate_possibili = [
    timedelta(minutes=d_min),
    timedelta(minutes=d_med),
    timedelta(minutes=d_max)
]

# 3. Orari possibili di inizio intervento (T)
# Il decisore valuta due opzioni: iniziare alle 08:00 o alle 10:00
orari_possibili_raw = ["08:00", "10:00", "12:00"]
orari_possibili = [parse_orario(h) for h in orari_possibili_raw]

# 4. Orari di partenza dei mezzi (S_i)
# Simuliamo voli/treni disponibili nella giornata
orari_partenza = {
    "Cuore":   [parse_orario("13:00"), parse_orario("15:00"), parse_orario("18:00")],
    "Polmoni": [parse_orario("13:30"), parse_orario("16:00")],
    "Rene Sx": [parse_orario("14:00"), parse_orario("18:00")],
    "Rene Dx": [parse_orario("14:15"), parse_orario("18:15")],
    "Fegato":  [parse_orario("15:00"), parse_orario("20:00")]
}

# --- ESECUZIONE ALGORITMO ---

valore_funzione_obiettivo, migliore_orario_inizio_intervento, tempo_di_esecuzione = robust_organ_scheduler(
    orari_possibili_inizio_intervento=orari_possibili,
    durate_possibili_intervento=durate_possibili,
    organi=organi,
    orari_partenza=orari_partenza,
    pesi=pesi
)

# --- VISUALIZZAZIONE RISULTATI ---

print(f"--- RISULTATO OTTIMIZZAZIONE ROBUSTA ---")
if migliore_orario_inizio_intervento:
    print(f"Orario Inizio Ottimo (t_start*): {migliore_orario_inizio_intervento.strftime('%H:%M')}")
    print(f"Valore Minimax (Ritardo Pesato): {valore_funzione_obiettivo:.2f}")
    print(f"Tempo di calcolo: {tempo_di_esecuzione.total_seconds():.6f} sec")
else:
    print("Nessuna soluzione ammissibile trovata (tutti gli scenari portano a ritardi infiniti).")

--- RISULTATO OTTIMIZZAZIONE ROBUSTA ---
Orario Inizio Ottimo (t_start*): 10:00
Valore Minimax (Ritardo Pesato): 18300.00
Tempo di calcolo: 0.000051 sec


## 4. Generazione di Scenari per l'Analisi Sperimentale

Per soddisfare il requisito di **misurazione sperimentale dei tempi di esecuzione** è necessario eseguire l'algoritmo su istanze di dimensioni crescenti.

Implementiamo una funzione `genera_scenario_test` che crea dataset sintetici parametrizzati:
* **Input:** Numero di orari di inizio intervento possibili ($|\mathbf{T}|$) e numero di partenze per organo ($|\mathbf{S}_i|$) e lista degli organi coinvolti.
* **Output:** Insiemi di orari (interventi e di partenza) generati casualmente ma coerenti temporalmente (distribuiti nelle 24 ore odierne).

Questo ci permetterà di stressare l'algoritmo variando la cardinalità di $\mathbf{T}$ da poche decine a migliaia di elementi.

In [11]:
import random
from datetime import datetime, date, timedelta
from typing import List, Dict, Tuple

def genera_scenario_test(
    num_orari_inizio_intervento: int,
    num_partenze_per_organo: int,
    lista_organi: List[str]
) -> Tuple[List[datetime], Dict[str, List[datetime]]]:
    """
    Genera un dataset sintetico per testare la scalabilità dell'algoritmo.
    Tutti gli orari (interventi e di partenza) sono confinati nelle 24 ore odierne.

    Args:
        num_orari_inizio_intervento: Cardinalità dell'insieme T.
        num_partenze_per_organo: Cardinalità di S_i (uguale per tutti gli organi).
        lista_organi: Lista degli organi da considerare nel test.

    Returns:
        T (list): Lista di datetime per gli orari di inizio (ordinati).
        S (dict): Dizionario con liste di datetime per le partenze di ogni organo (ordinate).
    """
    base_date = date.today()
    # Partiamo dalla mezzanotte di oggi
    base_time = datetime.combine(base_date, datetime.min.time())

    # 1. Generazione T (Orari inizio possibili) - CASUALE
    T = []
    # Finestra temporale di 24 ore (1440 minuti)
    finestra_giornaliera_minuti = 24 * 60

    for _ in range(num_orari_inizio_intervento):
        # Genera un minuto casuale tra 0 e 1440
        minuti_random = random.randint(0, finestra_giornaliera_minuti)
        T.append(base_time + timedelta(minutes=minuti_random))

    # Ordiniamo cronologicamente gli orari di input
    T.sort()

    # 2. Generazione S (Orari partenze mezzi) per gli organi
    S = {}

    for organo in lista_organi:
        orari_partenze = []
        # Generiamo orari casuali nelle STESSE 24 ore
        for _ in range(num_partenze_per_organo):
            minuti_random = random.randint(0, finestra_giornaliera_minuti)
            orari_partenze.append(base_time + timedelta(minutes=minuti_random))

        # Ordiniamo cronologicamente gli orari di partenza
        S[organo] = sorted(orari_partenze)

    return T, S

# Test immediato della funzione
num_orari_inizio_intervento = 10
num_partenze_per_organo = 13
lista_organi_input = ["Cuore", "Polmoni", "Rene Sx", "Rene Dx", "Fegato"]

T_esempio, S_esempio = genera_scenario_test(
    num_orari_inizio_intervento,
    num_partenze_per_organo,
    lista_organi_input
)

# Verifica T (Orari Inizio)
print(f"Generati {len(T_esempio)} orari di inizio.")
print(f"Esempio T (tutti i {len(T_esempio)} orari generati): {[t.strftime('%H:%M') for t in T_esempio]}")

# Verifica S (Esempio per un organo, es. Cuore)
print(f"\nGenerati orari di partenza per: {list(S_esempio.keys())}")
if "Cuore" in S_esempio:
    print(f"Esempio partenze 'Cuore': {[t.strftime('%H:%M') for t in S_esempio['Cuore']]}")
if "Fegato" in S_esempio:
    print(f"Esempio partenze 'Fegato': {[t.strftime('%H:%M') for t in S_esempio['Fegato']]}")

Generati 10 orari di inizio.
Esempio T (tutti i 10 orari generati): ['00:11', '04:23', '06:18', '09:23', '13:30', '14:50', '17:46', '19:23', '20:39', '22:08']

Generati orari di partenza per: ['Cuore', 'Polmoni', 'Rene Sx', 'Rene Dx', 'Fegato']
Esempio partenze 'Cuore': ['02:01', '08:16', '10:03', '10:46', '11:02', '13:18', '15:53', '15:54', '20:50', '20:51', '22:19', '22:35', '23:49']
Esempio partenze 'Fegato': ['00:35', '01:33', '01:57', '02:18', '02:32', '02:49', '07:02', '10:39', '18:08', '19:56', '21:45', '21:50', '23:34']


## 5. Esecuzione del Benchmark Sperimentale

Eseguiamo ora lo stress-test dell'algoritmo.
Manteniamo costanti i parametri di incertezza ($|\mathbf{D}|=3$) e il numero di partenze per organo ($|\mathbf{S}_i| = 50$), mentre facciamo variare la dimensione dell'insieme degli orari di inizio $\mathbf{T}$ da 50 a 5.000 elementi.

L'obiettivo è confermare empiricamente che il tempo di esecuzione cresce linearmente rispetto a $|\mathbf{T}|$.

In [14]:
import time

# --- CONFIGURAZIONE ESPERIMENTO ---

# Dimensioni dell'input T da testare (Scala logaritmica/crescente)
# Partiamo da 50 orari fino a 5000 per vedere la curva
test_sizes = [50, 100, 500, 1000]

# Parametri fissi per il test
NUM_PARTENZE_FIXED = 5000  # Simuliamo 50 voli disponibili per ogni organo
PESI_TEST = {"Cuore": 100, "Polmoni": 90, "Rene Sx": 20, "Rene Dx": 20, "Fegato": 20}
ORGANI_TEST = list(PESI_TEST.keys())

# Durate (min, med, max) standard
DURATE_TEST = [
    timedelta(minutes=180),
    timedelta(minutes=240),
    timedelta(minutes=300)
]

# Liste per salvare i risultati da graficare
risultati_tempi = []
risultati_dim = []

print(f"Avvio benchmark su {len(test_sizes)} scenari...")
print("-" * 75)
print(f"{'Dimensione T':<15} | {'Partenze per Organo':<20} | {'Tempo (sec)':<15}")
print("-" * 75)

for size in test_sizes:
    # 1. Generazione dello scenario randomico
    # Passiamo ORGANI_TEST come richiesto dalla nuova firma della funzione
    T_input, S_input = genera_scenario_test(
        num_orari_inizio_intervento=size,
        num_partenze_per_organo=NUM_PARTENZE_FIXED,
        lista_organi=ORGANI_TEST
    )

    # 2. Esecuzione Algoritmo
    # Nota: la funzione restituisce (valore, orario, tempo_exe)
    _, _, durata_esecuzione = robust_organ_scheduler(
        orari_possibili_inizio_intervento=T_input,
        durate_possibili_intervento=DURATE_TEST,
        organi=ORGANI_TEST,
        orari_partenza=S_input,
        pesi=PESI_TEST
    )

    # 3. Salvataggio dati per il grafico
    seconds = durata_esecuzione.total_seconds()
    risultati_tempi.append(seconds)
    risultati_dim.append(size)

    print(f"{size:<15} | {NUM_PARTENZE_FIXED:<20} | {seconds:.5f} sec")

print("-" * 75)
print("Benchmark completato.")

Avvio benchmark su 4 scenari...
---------------------------------------------------------------------------
Dimensione T    | Partenze per Organo  | Tempo (sec)    
---------------------------------------------------------------------------
50              | 50000                | 4.03178 sec
100             | 50000                | 9.71042 sec


KeyboardInterrupt: 